In [ ]:
import os
import numpy as np

from PIL import Image

%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


from sklearn.neighbors import KNeighborsClassifier
from joblib import dump, load
from collections import namedtuple

import torch

from get_feat_from_image import get_feat_from_image

# Get features from individual image

In [ ]:
Arguments = namedtuple('Arguments', ['arch',
                                     'patch_size',
                                     'pretrained_weights',
                                     'checkpoint_key',
                                     'image_path',
                                     'image_size',
                                     'output_dir',
                                     'threshold'
                                    ])

args = Arguments(arch = 'vit_small',
                 patch_size = 16,
                 pretrained_weights = '/path/to/the/pretrained/weights/checkpoint.pth',
                 checkpoint_key = "teacher",
                 image_path = '/path/to/the/image.jpg',
                 image_size = (1024, 1024),
                 output_dir = '/path/to/the/output/directory/',
                 threshold = 0.3
                )

In [ ]:
get_feat_from_image(args)

In [ ]:
IMAGE_PATH = '/path/to/the/image/created/after/the/model/processing/it/img.png'
image = Image.open(args.image_path)
plt.imshow(image)

# Load dim red model and features from single img

Load the dimensionality reduction model saved in order to process the features from the image

In [ ]:
dim_red_model = load('/path/to/the/dim_red_model')
scale_model = load('/path/to/the/scale_model')

Now load the singular image features!

In [ ]:
single_img_feats = torch.load('/path/to/the/feats.pt',map_location=torch.device('cpu'))
single_img_feats = scale_model.transform(single_img_feats)

In [ ]:
single_img_feats.shape

Now reduce the features dimensionality

In [ ]:
singular_sample = dim_red_model.transform(single_img_feats)
singular_sample.shape

## Load the clusters generated from the features

In [ ]:
clusters = np.load('/path/to/the/clusters.npy', allow_pickle=True)
vectors=clusters[()]['x']
labels=clusters[()]['y']

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(vectors, labels)

In [ ]:
def choose_random_rows(an_array, n_samples):
    number_of_rows = an_array.shape[0]
    random_indices = np.random.choice(number_of_rows, size=n_samples, replace=False)
    random_rows = an_array[random_indices, :]
    return random_rows

In [ ]:
X = choose_random_rows(vectors,5000)
n_components=X.shape[1]

In [ ]:
if n_components == 2:
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot()
    ax.scatter(X[:,0],X[:,1])
    ax.scatter(singular_sample[0,0],
               singular_sample[0,1],
               s=150
              )
elif n_components == 3:
    fig = plt.figure(figsize=(8,8))
    #ax = fig.add_subplot(projection='3d')
    ax = Axes3D(fig)
    ax.scatter(X[:,0], X[:,1], X[:,2])
    ax.scatter(
        singular_sample[0,0],
        singular_sample[0,1],
        singular_sample[0,2],
        s=60
    )

In [ ]:
label_predictions=neigh.predict(X)
core_samples_mask = np.zeros_like(label_predictions, dtype=bool)
core_samples_mask[label_predictions != -1] = True
n_noise_points = list(label_predictions).count(-1)

In [ ]:
# Black removed and is used for noise instead.
print_outliers=True
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
if n_components == 2:
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = label_predictions == k
        #class_member_mask = labels == k
        xy = X[class_member_mask & core_samples_mask]
        ax.plot(
            xy[:, 0],
            xy[:, 1],
            "o",
            markerfacecolor=tuple(col),
            markeredgecolor="k",
            markersize=14,
        )

        if print_outliers:
            xy = X[class_member_mask & ~core_samples_mask]
            ax.plot(
                xy[:, 0],
                xy[:, 1],
                "o",
                markerfacecolor=tuple(col),
                markeredgecolor="k",
                markersize=6,
            )
    
    ax.plot(
        singular_sample[0,0],
        singular_sample[0,1],
        "o",
        markerfacecolor="k",
        markersize=20,
    )

elif n_components == 3:
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = label_predictions == k
        #class_member_mask = labels == k
        xyz = X[class_member_mask & core_samples_mask]
        ax.scatter(
            xyz[:, 0],
            xyz[:, 1],
            xyz[:, 2],
            "o",
            c=[col],
            s=200,
        )

        if print_outliers:
            xyz = X[class_member_mask & ~core_samples_mask]
            ax.scatter(
                xyz[:, 0],
                xyz[:, 1],
                xyz[:, 2],
                "o",
                c=[col],
                s=100,
            )
    
    col = [1, 0, 0, 1]
    ax.scatter(
        singular_sample[0,0],
        singular_sample[0,1],
        singular_sample[0,2],
        "o",
        c=[col],
        s=300,
    )

string0 = "Estimated number of clusters: {}.\n"
string1 = "Estimated number of noise points during training: {}.\n"
string2 = "Number of noise points during inference: {}.\n"
string3 = "singular sample is of class: : {}.\n"
string = string0 + string1 + string2 #+ string3
plt.title(string .format(n_clusters_, n_noise_, n_noise_points, neigh.predict(singular_sample)[0]))

plt.show()

print(string .format(n_clusters_, n_noise_, n_noise_points, neigh.predict(singular_sample)[0]))